In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
tqdm.pandas(position=0,leave=True)
import textwrap
import random
import time 

import spacy
nlp = spacy.load("en_core_web_md")

from transformers import AutoModelForMaskedLM , AutoTokenizer
import torch
from NLP_prompt import Prompting

import warnings
warnings.filterwarnings("ignore")

model_path="bert-large-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_path)
prompting= Prompting(model=model_path)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/ec2-user/anaconda3/envs/test/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
# my_folder="s3://trident-retention-output/"

# #### Askunum text data ####
# askunum_text=pd.DataFrame()
# for year in [2018,2019,2020,2021,2022]:
#     new_data=pd.read_csv(os.path.join(my_folder,f"askunum_textbody_{year}"+".csv"))
#     askunum_text=pd.concat([askunum_text,new_data])
#     print("{:<15}{:<20,}".format(year,new_data.shape[0]))
    
# askunum_text.drop(['Unnamed: 0'],axis=1,inplace=True)
# askunum_text['unum_id']=askunum_text['unum_id'].astype(int).astype(str)
# askunum_text.sort_values(["unum_id","year","month","MessageDate"],inplace=True,ascending=True)

In [3]:
# start=time.time()
# df=askunum_text.groupby(["ParentId","account_id","unum_id"])['TextBody'].apply(lambda x: " ".join(x)).reset_index()
# df=df.drop_duplicates()
# end=time.time()
# print("It take {:.4f} second to group data".format(end-start))

In [4]:
# def truncation_text(X):
#     max_seq_length=tokenizer.model_max_length
#     truncated_input_ids=tokenizer(X,truncation=False,return_tensors="pt",add_special_tokens=False)['input_ids']
#     truncated_input_ids=truncated_input_ids[:,0:(max_seq_length - 2-2-6)].squeeze() ## 2 special tokens + 2 tokens for prefix-prompt: "email:"+ 6 tokens for post-prompt : ".this email has [MASK] sentiment"  
#     return tokenizer.decode(truncated_input_ids)

# df["truncated_TextBody"]=df["TextBody"].progress_apply(truncation_text)

# output_dir="s3://trident-retention-output/output/"
# df.to_pickle(os.path.join(output_dir,"askunum_text_truncation"))

In [ ]:
# start=time.time()
# output_dir="s3://trident-retention-output/output/"
# df=pd.read_pickle(os.path.join(output_dir,"askunum_text_truncation"))
# end=time.time()
# print("It take {:.4f} second to read data".format(end-start))

In [6]:
prefix_prompt="email:"
post_prompt=".this email has [MASK] sentiment"

threshold=prompting.compute_tokens_prob(prefix_prompt+post_prompt, token_list1=["positive","neutral"], token_list2= ["negative"],device=device)[0].item()
threshold

0.8184394836425781

In [ ]:
# def zero_shot_prompt(text):
#     text=prefix_prompt+text+post_prompt
#     prob=prompting.compute_tokens_prob(text, token_list1=["positive","neutral"], token_list2= ["negative"])[0].item()
#     return prob

# df1=df.sample(n=15000,random_state=102)
# df1["probability"]=df1["truncated_TextBody"].progress_apply(zero_shot_prompt)
# df1.sort_values("probability",ascending=True,inplace=True)


# df1.to_pickle(os.path.join(output_dir,"askunum_text_sentiment_v2"))

In [8]:
output_dir="s3://trident-retention-output/output/"
df=pd.read_pickle(os.path.join(output_dir,"askunum_text_sentiment_v2"))
df.head()

,ParentId,account_id,unum_id,TextBody,truncated_TextBody,probability
1617376,5003x000029QWWGAA4,0010c00002CokN3,850367362,unum covid-19 response - how to file a claim o...,unum covid - 19 response - how to file a claim...,0.188307
1906023,5003x00002Ei1r3AAB,001a000001WwWIC,189339935,gsc-mi kelsey caraway client name highlight in...,gsc - mi kelsey caraway client name highlight ...,0.246644
1185856,5003x000021UovvAAC,001a000001cqn74,293041571,please be advised that these employees wefurlo...,please be advised that these employees wefurlo...,0.349043
607755,5000c00001pJ123AAC,00130000018lyCQ,536776,pn 516239 the attached customerequest was rece...,pn 516239 the attached customerequest was rece...,0.349775
472949,5000c00001ki5XKAAY,001a000001q2FSx,528325726,darcy millelast day of employment 1/11/19. ple...,darcy millelast day of employment 1 / 11 / 19....,0.359935


In [12]:
def assign_label(prob):
    if prob<threshold:
        return "Negative"
    else:
        return "Positive or Neutral"
    
df1=df.copy()
df1["Predicted_sentiment"]=df1["probability"].progress_apply(assign_label)

100%|██████████| 15000/15000 [00:00<00:00, 1120433.11it/s]


In [13]:
df1.head(2)

,ParentId,account_id,unum_id,TextBody,truncated_TextBody,probability,Predicted_sentiment
1617376,5003x000029QWWGAA4,0010c00002CokN3,850367362,unum covid-19 response - how to file a claim o...,unum covid - 19 response - how to file a claim...,0.188307,Negative
1906023,5003x00002Ei1r3AAB,001a000001WwWIC,189339935,gsc-mi kelsey caraway client name highlight in...,gsc - mi kelsey caraway client name highlight ...,0.246644,Negative


In [15]:
df1["Predicted_sentiment"].value_counts(dropna=False)

Positive or Neutral    14174
Negative                 826
Name: Predicted_sentiment, dtype: int64

In [19]:
df2=df1.iloc[0:100]
df2=df2.append(df1.iloc[-100:])
df2["Predicted_sentiment"].value_counts(dropna=False)

Negative               100
Positive or Neutral    100
Name: Predicted_sentiment, dtype: int64

In [24]:
df2.drop("truncated_TextBody",axis=1,inplace=True)

In [26]:
df2.to_pickle(os.path.join(output_dir,"askunum_email_sample"))

In [25]:
df2.head()

,ParentId,account_id,unum_id,TextBody,probability,Predicted_sentiment
1617376,5003x000029QWWGAA4,0010c00002CokN3,850367362,unum covid-19 response - how to file a claim o...,0.188307,Negative
1906023,5003x00002Ei1r3AAB,001a000001WwWIC,189339935,gsc-mi kelsey caraway client name highlight in...,0.246644,Negative
1185856,5003x000021UovvAAC,001a000001cqn74,293041571,please be advised that these employees wefurlo...,0.349043,Negative
607755,5000c00001pJ123AAC,00130000018lyCQ,536776,pn 516239 the attached customerequest was rece...,0.349775,Negative
472949,5000c00001ki5XKAAY,001a000001q2FSx,528325726,darcy millelast day of employment 1/11/19. ple...,0.359935,Negative
